# RAG Exercise Solution

In [2]:
import dotenv

from agents import Agent, Runner, trace, function_tool

import chromadb

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [3]:
# We populated the RAG with the data from the data/calories.csv file in the rag_setup.ipynb notebook

chroma_client = chromadb.PersistentClient(path="../chroma")
calories_db = chroma_client.get_collection(name="nutrition_db")
nutrition_qna_db = chroma_client.get_collection(name="nutrition_qna")

In [4]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function too look up calorie information.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = calories_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)


@function_tool
def nutrition_qna_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function too ask a question about nutrition.

    Args:
        query: The question to ask
        max_results: The maximum number of results to return.

    Returns:
        A string containing the question and the answer related to the query.
    """

    results = nutrition_qna_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        formatted_results.append(doc)

    return "Related answers to your question:\n" + "\n".join(formatted_results)

In [7]:
# Creamos el agente ahora con 2 tools
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information and nutrtion advice.
    You give concise answers.
    
    If you need to look up calorie information, use the calorie_lookup_tool.
    If are asked a question about nutrition, always use the nutrition_qna_tool first to see if there is an answer in the knowledge base.
    """,
    tools=[calorie_lookup_tool, nutrition_qna_tool],
)

In [8]:
with trace("Nutrition Assistant with Nutrition and Calorie RAG"):
    result = await Runner.run(
        calorie_agent,
        "What are the best meal choices for pregnant women and how many calories do they have?",
    )
    print(result.final_output)

Here are key best meal choices for pregnancy and approximate calories:

- Focus on nutrient-dense foods daily:
  - Iron-rich: lean beef, poultry, beans, lentils, fortified cereals, spinach. Pair with vitamin C (citrus) to help absorption.
  - Folate: leafy greens, legumes, citrus, fortified grains.
  - Calcium: dairy or fortified alternatives; yogurt, cheese, milk.
  - Protein: lean meats, fish low in mercury, eggs, tofu, dairy, legumes.
  - Whole grains, fruits, vegetables, healthy fats (olive oil, avocado, nuts).

- Calorie guidance: aim for about 200–300 extra calories per day (some guidance suggests 200 calories extra; many recommendations use 300 in later pregnancy). Use this as a baseline and adjust for your activity and physician’s advice.

- Sample meal ideas (approximate calories):
  - Breakfast: yogurt parfait with Greek yogurt, berries, oats, and a drizzle of almonds (~350–420 kcal).
  - Lunch: quinoa salad with chickpeas, mixed veggies, feta, and olive oil (~500–600 kcal).
